# Script for training the ML-Models

Get datasets

In [ ]:
from os import listdir
from os.path import isfile, join

dataset_loc = 'training-data/'

datasets_names = [f for f in listdir(dataset_loc) if isfile(join(dataset_loc, f))]

Build+Train ML-Models:

In [ ]:
from flaml import AutoML
import pandas as pd
from datetime import datetime

automl_models = dict()

for dataset_name in datasets_names:
  df = pd.read_csv(dataset_loc + dataset_name)
  dataset_name_short = dataset_name[0:-9]
  X_train, y_train = df.iloc[:, :-1], df.iloc[:, -1]

  automl_settings = {
    "task": "classification",
    "time_budget": 2*60,
    "metric": 'accuracy',
    "log_file_name": 'logs/{}{}.log'.format(dataset_name, datetime.now())    
  }

  automl = AutoML()
  automl.fit(X_train, y_train, **automl_settings)

  automl_models[dataset_name_short] = {
    'model': automl
  }

Save-AutoML Models

In [ ]:
import pickle

with open(r'../app/ml-models.pickle', "wb") as output_file:
  pickle.dump(automl_models, output_file, -1)